<a href="https://colab.research.google.com/github/roshinisanikop/Mitigating_Bias_in_Facial_Recognition_using_Seldonian_Framework/blob/main/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

Download all the dependencies and the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

FOLDERNAME = 'ColabNotebooks'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive/


In [ ]:
# Import PyTorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
# Note: your PyTorch version shouldn't be lower than 1.10.0 and torchvision version shouldn't be lower than 0.11
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

from datasets import DatasetDict, Dataset
import pandas as pd
from datasets import load_dataset

In [ ]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Prepping Dataset

In [ ]:
ds = load_dataset("HuggingFaceM4/FairFace", "0.25")

In [ ]:
from datasets import DatasetDict, Dataset
import pandas as pd

def create_balanced_ds( dataset ):
  df_train = ds['train'].to_pandas()
  df_val = ds['validation'].to_pandas()

  # Step 1: Find the minimum count among all races
  min_count_train = df_train['race'].value_counts().min()
  min_count_val = df_val['race'].value_counts().min()

  # Step 2: Randomly sample data for each race
  balanced_df_train = df_train.groupby('race').apply(lambda x: x.sample(min_count_train, random_state=42)).reset_index(drop=True)
  balanced_df_val = df_val.groupby('race').apply(lambda x: x.sample(min_count_val, random_state=42)).reset_index(drop=True)

  # Remove service_test
  balanced_df_train = balanced_df_train.iloc[:, 0:4]
  balanced_df_val = balanced_df_val.iloc[:, 0:4]

  return balanced_df_train, balanced_df_val

In [ ]:
from PIL import Image
import io
import numpy as np

def process_rgb_image(image_dict):
    try:
        # Load image from bytes
        image_bytes = image_dict['bytes']
        image = Image.open(io.BytesIO(image_bytes))  # No conversion to grayscale here

        # Resize image to a fixed size (e.g., 48x48)
        image = image.resize((48, 48))

        # Convert to NumPy array
        image_array = np.array(image, dtype=np.float32)

        # Normalize pixel values to range [0, 1]
        image_array /= 255.0

        return image_array
    except Exception as e:
        print(f"Error processing image: {e}")
        return None